# Hello PyOpenCL

In [2]:
import pyopencl as cl
import numpy as np
import numpy.linalg as la

mf = cl.mem_flags

This notebook demonstrates the simplest PyOpenCL workflow that touches all essential pieces:

* Data transfer
* Kernel compilation
* Execution

In [3]:
a = np.random.rand(50000).astype(np.float32)

Now create a context.

In [4]:
#clear
ctx = cl.create_some_context()

queue = cl.CommandQueue(ctx)

Now allocate a buffer. `Buffer(context, flags, size=None, hostbuf=None)`

In [5]:
#clear
a_buf = cl.Buffer(ctx, mf.READ_WRITE, size=a.nbytes)

Then transfer data:

In [7]:
#clear
cl.enqueue_copy(queue, a_buf, a)

Here's our kernel source code:

In [8]:
prg = cl.Program(ctx, """
    __kernel void twice(__global float *a)
    {
      int gid = get_global_id(0);
      a[gid] = 2*a[gid];
    }
    """).build()

Run the kernel.

In [9]:
#clear
prg.twice(queue, a.shape, None, a_buf)

Copy the data back.

In [10]:
#clear
result = np.empty_like(a)

cl.enqueue_copy(queue, result, a_buf)

Check the result.

In [11]:
#clear
print(la.norm(result - 2*a), la.norm(a))

0.0 129.302
